## More Lions is All You Need

running scoring and voting using `llama3` model via `ollama` service

In [1]:
system = "you are a helpful assistant."
instruct = "think step by step, concisely yet comprehensively explain the balance sheet. return around 100 words"

The goal is to find the best answer from 5 generations of the same questions

In [2]:
import lionagi as li

In [3]:
durations = []


async def explain():

    imodel = li.iModel(model="llama3", provider="ollama")
    branch = li.Branch(system=system, imodel=imodel)

    await branch.chat(instruction=instruct)
    return branch

In [4]:
# write reports

reports = []
for i in range(5):
    writer = await explain()
    report = writer.last_response.response
    report = {"id": i, "report": report}
    reports.append(report)
    print(f"Report No.{i+1} finished.")

Report No.1 finished.
Report No.2 finished.
Report No.3 finished.
Report No.4 finished.
Report No.5 finished.


### Get a group of critics

In [5]:
system = """
    as a highly critical professor, score an answer, only return a 
    positive integer
"""

instruction = """
    meticulously and critically give an integer score between 0-10 on the 
    following explaination of balance sheet.
"""

In [6]:
import numpy as np
from lionagi.libs import to_num


async def scoring(context):
    imodel = li.iModel(model="llama3", provider="ollama")
    branch = li.Branch(system=system, imodel=imodel)
    out_ = await branch.chat(instruction=instruction, context=context)

    # let's convert the output to a number
    try:
        out_ = to_num(out_, upper_bound=10, lower_bound=0, num_type=int)
    except:
        out_ = -1
    return out_

method 1

In [7]:
async def group_scoring(context):
    outs = []

    # we will score each item 5 times
    # if you have high enough rate limits / compute,
    # you can modify the code using `li.alcall`, to concurrently run the scoring
    for _ in range(5):
        out_ = await scoring(context)
        outs.append(out_)

    # we need to exlude the -1 values
    outs = [i for i in outs if i != -1]

    return np.mean(outs)

In [8]:
from time import time

start = time()

scores = []

# run scoring
for idx, i in enumerate(reports):
    score_ = await group_scoring(i)
    scores.append((score_, i))

duration = time() - start
print(f"Duration: {duration: .03f} seconds")

Duration:  125.544 seconds


In [9]:
# show results
scores = sorted(scores, key=lambda x: x[1]["id"])
for score_, report in scores:
    print(f"Report: No.{report['id']}, Score: {score_}")

Report: No.0, Score: 8.4
Report: No.1, Score: 8.0
Report: No.2, Score: 8.0
Report: No.3, Score: 8.0
Report: No.4, Score: 7.8


method 2 (recommend if use API)

use `li.alcall` if we use async functions, you can run the scoring concurrently, but in local envrionment it doesn't necessarily speed things up

In [10]:
start = time()


async def group_scoring2(context):

    async def _scoring(i):
        return await scoring(context)

    outs = await li.alcall(range(5), _scoring)
    outs = [i for i in outs if i != -1]

    return np.mean(outs)


async def score_report2(report):

    score_ = await group_scoring2(report)

    print("Scoring completed. Score:", score_)
    return (score_, report)


scores2 = await li.alcall(reports, score_report2)

duration = time() - start
print(f"Duration: {duration:.3f} seconds")

Scoring completed. Score: 7.6
Scoring completed. Score: 8.0
Scoring completed. Score: 8.2
Scoring completed. Score: 7.8
Scoring completed. Score: 8.2
Duration: 109.299 seconds


In [11]:
scores2 = sorted(scores2, key=lambda x: x[1]["id"])
for score_, report in scores2:
    print(f"Report: No.{report['id']}, Score: {score_}")

Report: No.0, Score: 7.6
Report: No.1, Score: 8.2
Report: No.2, Score: 8.2
Report: No.3, Score: 7.8
Report: No.4, Score: 8.0


### Reports

In [12]:
# rank scores
scores = sorted(scores, key=lambda x: x[0], reverse=True)

In [19]:
from IPython.display import display, Markdown

In [20]:
score, report = scores[0]

print(f"Score: {score}")
display(Markdown(f"**Report:**\n\n{report['report']}"))

Score: 8.4


**Report:**

Here's a concise and comprehensive explanation of the balance sheet:

A balance sheet is a snapshot of a company's financial position at a specific point in time. It's divided into three main sections: Assets, Liabilities, and Equity.

**Assets**: What the company owns or controls, such as:

* Cash and cash equivalents
* Accounts receivable (debts owed to the company)
* Inventory (goods or materials held for sale)
* Property, plant, and equipment (PP&E)
* Intangible assets (patents, trademarks, etc.)

**Liabilities**: What the company owes or is responsible for, including:

* Accounts payable (debts owed by the company)
* Short-term loans
* Long-term debt
* Accrued expenses

**Equity**: The company's net worth, consisting of:

* Common stock
* Preferred stock
* Retained earnings (profits reinvested)

The balance sheet aims to provide a fair and accurate picture of the company's financial health.

In [21]:
score, report = scores[1]

print(f"Score: {score}")
display(Markdown(f"**Report:**\n\n{report['report']}"))

Score: 8.0


**Report:**

Here's a concise and comprehensive explanation of the balance sheet:

**Balance Sheet: A Snapshot of Your Company's Financial Situation**

A balance sheet is a financial statement that provides a snapshot of your company's financial situation at a specific point in time (usually the end of an accounting period). It summarizes your business's assets, liabilities, and equity.

**Assets (left side):**

* Cash, accounts receivable, inventory, property, equipment, etc.
* Represent what your business owns or is owed.

**Liabilities (right side):**

* Accounts payable, loans, mortgages, taxes owed, etc.
* Represent what your business owes to others.

**Equity (bottom):**

* Share capital, retained earnings, dividends, etc.
* Represents the ownership interest in your business.

The balance sheet ensures that the value of assets equals the sum of liabilities and equity.

In [22]:
score, report = scores[2]

print(f"Score: {score}")
display(Markdown(f"**Report:**\n\n{report['report']}"))

Score: 8.0


**Report:**

Here's a concise explanation of the balance sheet:

The Balance Sheet is a financial statement that presents a company's financial position at a specific point in time. It's divided into three main sections: Assets, Liabilities, and Equity.

**Assets**: What a company owns or controls, such as cash, accounts receivable, inventory, property, and investments. ($1,000,000)

**Liabilities**: What a company owes to others, including accounts payable, loans, and taxes owed. ($800,000)

**Equity**: The ownership interest in the business, representing the amount of capital invested by shareholders. ($200,000)

The Balance Sheet is "balanced" because the total value of Assets equals the sum of Liabilities and Equity: $1,000,000 = $800,000 + $200,000. This shows that a company's financial position is stable and consistent with its income statement and cash flow statement.

In [23]:
score, report = scores[3]

print(f"Score: {score}")
display(Markdown(f"**Report:**\n\n{report['report']}"))

Score: 8.0


**Report:**

Here's a concise and comprehensive explanation of a balance sheet:

A balance sheet is a financial statement that presents a company's financial position at a specific point in time. It's divided into three main sections: Assets, Liabilities, and Equity.

**Assets**: What the company owns or controls, such as cash, accounts receivable, inventory, property, and equipment. Total assets = $1,000,000

**Liabilities**: What the company owes to others, including accounts payable, loans, and taxes owed. Total liabilities = $500,000

**Equity**: The company's net worth, representing the owners' stake in the business. Equity is calculated by subtracting liabilities from assets: Total equity = $1,000,000 - $500,000 = $500,000

The balance sheet shows that the company has a total of $1,000,000 in assets and $500,000 in liabilities, resulting in a net worth (equity) of $500,000.

In [24]:
score, report = scores[4]

print(f"Score: {score}")
display(Markdown(f"**Report:**\n\n{report['report']}"))

Score: 7.8


**Report:**

Here's a concise and comprehensive explanation of the balance sheet:

**What is it?**: A snapshot of a company's financial position at a specific point in time.

**Components:**

1. **Assets**: What the company owns, such as:
	* Cash
	* Accounts receivable (amounts owed to the company)
	* Inventory (goods or materials for sale)
	* Property, plant, and equipment (long-term assets)
2. **Liabilities**: Amounts the company owes, including:
	* Accounts payable (amounts owed by the company)
	* Loans or debt
3. **Equity**: The company's net worth, representing:
	* Shareholders' investments
	* Retained earnings (profits reinvested)

**Equation:** Assets = Liabilities + Equity

This balance ensures that the company's financial situation is stable and consistent, providing stakeholders with a clear picture of its financial health.